# **INF8460 A20 Project: Open-domain questions answering**

<br>

Equipe 8:


*   Cedric Sadeu 
*   Mamoudou Sacko 
*   Oumayma Messoussi

<br>

---

<br>

In [ ]:
!pip install sent2vec transformers pytorch-pretrained-bert -q

     |████████████████████████████████| 1.4MB 7.8MB/s 
     |████████████████████████████████| 133kB 24.6MB/s 
     |████████████████████████████████| 890kB 23.6MB/s 
     |████████████████████████████████| 2.9MB 43.2MB/s 
     |████████████████████████████████| 133kB 54.8MB/s 
     |████████████████████████████████| 7.1MB 51.2MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
ERROR: botocore 1.19.36 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


---

<br>

## **1. Map Google Drive**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/INF8460/Project/')

Mounted at /content/drive


---

<br>

## **2. Import**

- hub.py: acts as a "controller" and groups together all pipeline sections and calls the scripts listed below
- create_embeddings.py: contains functions to construct vector representations (TF-IDF, GloVe, BERT)
- data_handling.py: data loading and preprocessing
- bert.py: answer extraction functions for BERT

In [ ]:
from hub import *
import torch
from time import time
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


---

<br>

## **3. Load Data**


In [ ]:
corpus_data = load_data("corpus", 0)
vals_data = load_data("validation", force_refresh = 0)
test_data = load_data("test", force_refresh = 0)

corpus_ids = corpus_data[0]
vals_ids = corpus_data[0]
test_ids = test_data[0]

corpus_paragraphs = corpus_data[1]
vals_questions = vals_data[1]
test_questions = test_data[1]

del corpus_data
del vals_data
del test_data

In [ ]:
print("- passage %d: %s\n- val question %d: %s" % (corpus_ids[5], corpus_paragraphs[5], vals_ids[5], vals_questions[5]))

- passage 5: The descendants of Rollo's Vikings and their Frankish wives would replace the Norse religion and Old Norse language with Catholicism (Christianity) and the Gallo-Romance language of the local people, blending their maternal Frankish heritage with Old Norse traditions and customs to synthesize a unique "Norman" culture in the north of France. The Norman language was forged by the adoption of the indigenous langue d'oïl branch of Romance by a Norse-speaking ruling class, and it developed into the regional language that survives today.
- val question 5: Who were the two most powerful nations in the 1860's?


---

<br>

## **4. Prepocessing Data**

In [ ]:
corpus_text = prepoc_data(corpus_paragraphs)
val_text = prepoc_data(vals_questions)
test_text = prepoc_data(test_questions)

# del corpus_paragraphs
# del test_questions
# del vals_questions

In [ ]:
print("- passage %d: %s\n- val question %d: %s" % (corpus_ids[5], corpus_text[5], vals_ids[5], val_text[5]))

- passage 5: the descendant of rollo viking and their frankish wife would replace the norse religion and old norse language with catholicism christianity and the gallo romance language of the local people blending their maternal frankish heritage with old norse tradition and custom to synthesize unique norman culture in the north of france the norman language wa forged by the adoption of the indigenous langue oïl branch of romance by norse speaking ruling class and it developed into the regional language that survives today
- val question 5: who were the two most powerful nation in the


---

<br>

## **5. Vector representations**




> ##### TFIDF



In [ ]:
representation_name = "tfidf"
corpus_emmbeding , test_emmbeding = get_embedding(corpus_text, test_text, representation_name)
corpus_emmbeding , val_emmbeding = get_embedding(corpus_text, val_text, representation_name)

del corpus_text
del test_text
del val_text

> ##### Glove

In [ ]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip # larger vocab
!unzip glove.42B.300d.zip

In [ ]:
representation_name = "glove"
corpus_emmbeding , val_emmbeding = get_embedding(corpus_text, val_text, representation_name)
corpus_emmbeding , test_emmbeding = get_embedding(corpus_text, test_text, representation_name)

del corpus_text
del val_text
del test_text

> ##### Bert

In [ ]:
representation_name = "bert"
corpus_emmbeding , val_emmbeding = get_embedding(corpus_text, val_text, representation_name)
corpus_emmbeding , test_emmbeding = get_embedding(corpus_text, test_text, representation_name)

del corpus_text
del val_text
del test_text

---

<br>

## **6. Ordonnancement**

In [ ]:
# we computed the ranking lists on batches of 2500 questions to parallelize processing on multipple Colab instances

start = time()
# ranking_list = get_ranking_list(corpus_ids, corpus_emmbeding, vals_ids[:2500], val_emmbeding[:2500], metrique=cosine, top=25)
ranking_list = get_ranking_list(corpus_ids, corpus_emmbeding, test_ids[2500:5000], test_emmbeding[2500:5000], metrique=cosine, top=25)
print(time() - start)

100%|██████████| 2500/2500 [9:22:28<00:00, 13.50s/it]

33755.915775060654


In [ ]:
# save ranking results in csv file

import csv

w = csv.writer(open("/content/drive/My Drive/Colab Notebooks/INF8460/Project/output/2500-5000ranking_test_tfidf_ngrams_15000_cosine_top25.csv", "w"))
w.writerow(['id', 'top_25', 'top_10'])
for key, val in ranking_list.items():
    w.writerow([key, val, val[:10]])

---

<br>

## **7. Answer extraction**

**with ELECTRA pretrained on SQuAD2.0**

In [ ]:
from tqdm import tqdm

def get_answers(nlp, paragraphs_id, paragraphs, questions_id, questions, ranking_list):

    questions_answers = {}

    for question_id in tqdm(ranking_list):
        question = questions[question_id]
        top_paragraphs_ids = ranking_list[question_id]
        top_paragraphs_text = getParagraph(top_paragraphs_ids, paragraphs_id, paragraphs)

        questions_answers[question_id] = []
        for j, paragraph in enumerate(top_paragraphs_text):
            QA_input = { 'question': question, 'context': paragraph }
            answers = nlp(QA_input)
            questions_answers[question_id].append((top_paragraphs_ids[j], answers))

        questions_answers[question_id] = sorted(questions_answers[question_id], key = lambda i: i[1]['score'], reverse=True)

    return questions_answers

In [ ]:
# read ranking results from csv file

import pandas as pd

df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/INF8460/Project/output/ranking_val_tfidf_ngrams_15000_cosine_top25.csv")
df.head()

ranking_list = {}
for id, topn in zip(df['id'], df['top_25']):
    ranking_list[id] = [int(x) for x in topn[1:len(topn)-1].split(', ')]

print(len(ranking_list))
print(ranking_list[0])
del df

10000
[79235, 41722, 79832, 17123, 52070, 36258, 15269, 2064, 58498, 63237, 17153, 17134, 43464, 74862, 11102, 53230, 8154, 62487, 17159, 54677, 79717, 17109, 42345, 17133, 60145]


In [ ]:
# load ELECTRA pretrained model on SQuAD2.0

from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import csv
import json

model_name = "deepset/electra-base-squad2"

nlp = pipeline('question-answering', model=model_name, tokenizer=model_name, device=0)

answers = get_answers(nlp, corpus_ids, corpus_paragraphs, test_ids, test_questions, ranking_list)
# answers = get_answers(nlp, corpus_ids, corpus_paragraphs, vals_ids, vals_questions, ranking_list)

100%|██████████| 10000/10000 [1:28:11<00:00,  1.89it/s]


In [ ]:
# save answers to csv file along with scores

w = csv.writer(open("/content/drive/My Drive/Colab Notebooks/INF8460/Project/output/answers_val_tfidf_ngrams_15000_cosine_top25.csv", "w"))
w.writerow(['id', 'paragraph_id', 'answer', 'score'])
for key, val in answers.items():
    par = val[0][0]
    ans = val[0][1]['answer']
    score = val[0][1]['score']
    w.writerow([key, par, ans, score])

tmp = pd.read_csv("/content/drive/My Drive/Colab Notebooks/INF8460/Project/output/answers_val_tfidf_ngrams_15000_cosine_top25.csv")

**with ELECTRA fine-tuned on our dataset**

In [ ]:
from tqdm import tqdm
import nltk

nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words("english"))

from bs4 import BeautifulSoup
def removeHtml(texts):
    return BeautifulSoup(texts, "lxml").text


def get_answers_model(model, tokenizer, paragraphs_id, paragraphs, questions_id, questions, ranking_list):

    questions_answers = {}

    for question_id in tqdm(ranking_list):
        question = questions[question_id]
        top_paragraphs_ids = ranking_list[question_id][:15]
        top_paragraphs_text = getParagraph(top_paragraphs_ids, paragraphs_id, paragraphs)

        questions_answers[question_id] = []
        for j, paragraph in enumerate(top_paragraphs_text):
            answers = getAnsers(question, removeHtml(paragraph), model, tokenizer)
            questions_answers[question_id].append((top_paragraphs_ids[j], answers))

        questions_answers[question_id] = sorted(questions_answers[question_id], key = lambda i: i[1]['score'], reverse=True)
        print(questions_answers[question_id])

    return questions_answers

def getAnsers(question, text, model, tokenizer):

    inputs = tokenizer(question, text, return_tensors='pt', padding=True, truncation=True).to(device)
    start_positions = torch.tensor([1]).to(device)
    end_positions = torch.tensor([3]).to(device)

    outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
    loss = outputs.loss
    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    input_ids = inputs["input_ids"].tolist()[0]

    answer_start = torch.argmax(start_scores)  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(end_scores) + 1  # Get the most likely end of answer with the argmax of the score
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    score = start_scores.max().item()
    if answer in stopwords:
      score = 0

    reponse_data = {'score': score, 'start': start_scores.max().item(), 'end': end_scores.max().item(), 'answer': answer}
    return reponse_data

In [ ]:
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizerFast, pipeline
import csv

model_name = "deepset/electra-base-squad2"
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name, device=0) # this loads ELECTRA pre-trained on SQuAD2.0

# model_electra_fine_tuned_custom_data_v3.pt: fine-tuned on our corpus for 1 epoch
# model_electra_fine_tuning_custom_data_v2.pt: fine-tuned on our corpus for 2 epochs
# model_bert_fine_tuned_squad.pt: BERT large model fine-tuned on SQuAD2.0

output_path = "/content/drive/My Drive/Colab Notebooks/INF8460/Project/output/model_electra_fine_tuned_custom_data_v3.pt" 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = nlp.model.to(device)
model.load_state_dict(torch.load(output_path)) # this replaces ELECTRA with our fine-tuned ELECTRA model
model.eval()

nlp.model = model.to(device)

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

100%|██████████| 2475/2475 [4:19:49<00:00,  6.30s/it]


100% done



In [ ]:
answers = get_answers_model(model, tokenizer, corpus_ids, corpus_paragraphs, vals_ids, vals_questions, ranking_list)

  0%|          | 1/10000 [00:00<59:22,  2.81it/s]

[(10826, {'score': 2.0902645587921143, 'start': 2.0902645587921143, 'end': 2.467841625213623, 'answer': '25 million hectares in 2011. this was 69 %'}), (47824, {'score': 1.6006709337234497, 'start': 1.6006709337234497, 'end': 2.207653284072876, 'answer': 'texas'}), (10827, {'score': 1.2881966829299927, 'start': 1.2881966829299927, 'end': 1.32919442653656, 'answer': '10. 6 million hectares'}), (63154, {'score': 0.537036120891571, 'start': 0.537036120891571, 'end': 1.2400113344192505, 'answer': '17. 31 million bales'}), (10837, {'score': 0.2800346612930298, 'start': 0.2800346612930298, 'end': 0.7817607522010803, 'answer': '$ 2. 1 billion'}), (10823, {'score': 0.04339395463466644, 'start': 0.04339395463466644, 'end': 0.6141999363899231, 'answer': 'cotton'}), (10830, {'score': 0, 'start': -0.07834015786647797, 'end': 0.9666969776153564, 'answer': 'most'}), (10821, {'score': 0, 'start': -1.3546779155731201, 'end': -1.0908753871917725, 'answer': 'by'}), (10839, {'score': 0, 'start': -0.48840

  0%|          | 2/10000 [00:00<58:50,  2.83it/s]

[(6915, {'score': 3.857879638671875, 'start': 3.857879638671875, 'end': 3.2049388885498047, 'answer': 'zythology'}), (79723, {'score': 2.3442201614379883, 'start': 2.3442201614379883, 'end': 2.3960633277893066, 'answer': 'schlitz beer'}), (6910, {'score': 1.5258442163467407, 'start': 1.5258442163467407, 'end': 2.0363547801971436, 'answer': 'gilgamesh'}), (6916, {'score': 1.041983962059021, 'start': 1.041983962059021, 'end': 1.514721393585205, 'answer': 'brewing'}), (18259, {'score': 1.0348807573318481, 'start': 1.0348807573318481, 'end': 1.9227702617645264, 'answer': 'gilgamesh'}), (3610, {'score': 0, 'start': -1.4510279893875122, 'end': -0.734400749206543, 'answer': 'a'}), (6914, {'score': -0.17226286232471466, 'start': -0.17226286232471466, 'end': 0.06337710469961166, 'answer': 'freeze - distilling process'}), (6890, {'score': -0.4368699789047241, 'start': -0.4368699789047241, 'end': 0.22679635882377625, 'answer': 'brewing'}), (3580, {'score': -0.7273493409156799, 'start': -0.7273493

  0%|          | 3/10000 [00:01<57:51,  2.88it/s]

...

  7%|▋         | 654/10000 [03:43<52:56,  2.94it/s]

[(731, {'score': 2.700737476348877, 'start': 2.700737476348877, 'end': 2.8262243270874023, 'answer': 'foreign'}), (36143, {'score': 2.448918342590332, 'start': 2.448918342590332, 'end': 3.324420690536499, 'answer': 'rein'}), (10193, {'score': 2.3841845989227295, 'start': 2.3841845989227295, 'end': 3.2962944507598877, 'answer': 'second language'}), (19306, {'score': 1.4036368131637573, 'start': 1.4036368131637573, 'end': 1.2326687574386597, 'answer': 'bible'}), (4940, {'score': 1.0352849960327148, 'start': 1.0352849960327148, 'end': 1.236018419265747, 'answer': 'classical'}), (3810, {'score': 0.9206424951553345, 'start': 0.9206424951553345, 'end': 1.0463072061538696, 'answer': 'stops and affricates'}), (22603, {'score': 0.41698211431503296, 'start': 0.41698211431503296, 'end': 0.8372767567634583, 'answer': 'academic major'}), (4730, {'score': 0.0012337099760770798, 'start': 0.0012337099760770798, 'end': 0.7285211086273193, 'answer': 'spanish, portuguese, and italian'}), (55105, {'score'

  7%|▋         | 655/10000 [03:44<53:10,  2.93it/s]

[(17149, {'score': 4.0358781814575195, 'start': 4.0358781814575195, 'end': 4.2216105461120605, 'answer': 'star tv'}), (3995, {'score': 2.428636074066162, 'start': 2.428636074066162, 'end': 2.8877458572387695, 'answer': 'beidou - 1 is an experimental regional navigation system, which consists of four satellites ( three working satellites and one backup satellite ). the satellites themselves were based on the chinese dfh - 3 geostationary communications satellite'}), (36258, {'score': 2.3402047157287598, 'start': 2.3402047157287598, 'end': 2.0574216842651367, 'answer': 'hussain sagar lake'}), (43464, {'score': 1.980480670928955, 'start': 1.980480670928955, 'end': 1.2305246591567993, 'answer': 'deccan airways limited'}), (74862, {'score': 1.980480670928955, 'start': 1.980480670928955, 'end': 1.2305246591567993, 'answer': 'deccan airways limited'}), (35977, {'score': 1.8437360525131226, 'start': 1.8437360525131226, 'end': 2.52231502532959, 'answer': 'ethernet'}), (3783, {'score': 1.8091276

  7%|▋         | 656/10000 [03:44<53:11,  2.93it/s]

In [ ]:
# save extracted answers to csv file

import csv

w = csv.writer(open("/content/drive/My Drive/Colab Notebooks/INF8460/Project/output/answers_test_tfidf_15000_cosine_top25.csv", "w"))
w.writerow(['id', 'paragraph', 'answer'])
for key, val in answers.items():
    w.writerow([key, val[0][0], val[0][1]['answer']])